# Baseline solution

## Delete
The concept for this approach draws inspiration from how mass media handles offensive language. For instance, in TV programs and articles, swear words are often obscured or substituted with asterisks (`**`). Following a similar principle, goal of this baseline is to remove offensive words from the source text. 

To perform the baseline solution, I have created a dictionary containing the most frequently used swear words from the obscenity list taken from [here](https://github.com/surge-ai/profanity/tree/main). Using nltk library, I cleaned the reference sentences. Any words that match the words in that dictionary of swear words are then removed filtering out toxic language.


In [ ]:
%cd ..
from tqdm import tqdm
import pandas as pd
import requests
import io
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

In [2]:
# Load the dictionary of profanity words
def download_csv_and_save_column(url, file_path):
    # Download the CSV file using the link
    response = requests.get(url)
    
    # Load the CSV data into a DataFrame
    df = pd.read_csv(io.StringIO(response.text))
    
    # Save the dictionary to a text file (+ add some toxic words that not present in dictionary)
    profanity_vocab = set(df['canonical_form_1'].to_list() + ['idiot', 'stupid', 'fucking', 'nigga'])
    with open(file_path, 'w') as file:
        for word in profanity_vocab:
            file.write(word + '\n')
    

vocab_url = 'https://github.com/surge-ai/profanity/raw/main/profanity_en.csv'
vocab_path = './data/external/profanity_vocab.txt'
download_csv_and_save_column(vocab_url, vocab_path)

In [3]:
# Create the vocab of toxic words 
with open(vocab_path) as vocab_file:
        vocab = [line.strip().split()[0] for line in vocab_file.readlines()]

vocab[:6]

['gypsy', 'upskirt', 'tadger', 'tossing', 'shit', 'bean']

Since for this baseline training process is not required, I will perform it only on test set and then save the results for further evaluation.

In [4]:
df = pd.read_csv('./data/interim/test.csv')

source = df['reference'].to_list()
target = df['detox_reference'].to_list()

In [5]:
def delete_toxic_words(texts, vocab):
    """
    Deletes all toxic words from a pre-defined vocab in a list of source texts,
    and writes the resulting texts to an array.

    Args:
        texts (list): A list of strings containing the source texts to be cleaned.
        vocab (list): The list of obscene words.

    Returns:
        list: A list of cleaned texts with toxic words removed.
    """

    results = []

    for text in tqdm(texts, desc="Deleting toxic words"):
        # Tokenize the text into words
        text = word_tokenize(text)
        
        # Remove all toxic words from the text
        clean_text = ' '.join(
            [word for word in text if word.lower() not in vocab]
        )
        
        # Remove extra spaces before punctuation
        clean_text = re.sub(r'\s+([^\w\s])', r'\1', clean_text)
        
        results.append(clean_text)

    return results

In [6]:
result = delete_toxic_words(source, vocab=vocab)

Deleting toxic words: 100%|██████████| 20000/20000 [00:01<00:00, 14065.83it/s]


In [7]:
# Save the results to a CSV file
df['baseline_result'] = result

resulting_dataset_path = './data/interim/baseline_results.csv'
df.to_csv(resulting_dataset_path, index=False)

## Some examples

In [8]:
for i in [5, 15, 25, 50]:
    print('Source:', source[i])
    print('Baseline res:', result[i])
    print('Target:', target[i], '\n')

Source: I was there when he took a photo to your super penis.
Baseline res: I was there when he took a photo to your super.
Target: I was there when he took a picture next to your superbird. 

Source: This shit's temporary.
Baseline res: This's temporary.
Target: it's temporary. 

Source: why, you little bastard!
Baseline res: why, you little!
Target: Why you little! 

Source: when this vampire shit moves, people either start dating again, or we all get killed.
Baseline res: when this vampire moves, people either start dating again, or we all get killed.
Target: 'Cause once this vampire situation is over, folk gonna be back or we gonna be dead. 



However, this approach is not the perfect one as it may result in the loss of contextual information within the original toxic sentence.